<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create fictive clinical notes from Code set (DRG + ICD)

Code set are the raw classification data, we can extract from National database (Base nationale PMSI en France). They are made of 
* classification profile made of grouping variables from DRG records which are prepared with their frequency in the national database
    - age (class)
    - sexe
    - DRG (racine GHM)
    - Main diagnosis (ICD10) : cf
    - Hospitalization management type : cf
* diagnosis associated to each classification profile, extracted with their frequencies
* procedures associated to each classification profile, specialy for surgery and technical gestures, extracted with their frequencies

From thoses raw information we produce a coded clinical scenario which will be uses a seed.

This scenario is transformed into a detail prompt that will be given to a LLM for generation.
From the combinaision of primary and related diagnosis in French discharge abstract, we derived two notions :
* Primary diagnosis : host the notion of principal pathology, it is rather the primary diagnosis of the discharge abstract or the related diagnosis when it exists and that the primary diagnosis of the discharge abstract is from the chapter "Facteurs influant sur l’état de santé" of ICD10
* The Hospitalization management type is rather the term "Primary diagnosis" or the ICD-10 code of the related diagnosis when it exists


In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
import numpy as np
import datetime as dt

In [11]:
from utils import *

In [12]:
path_ref  = "referentials/"

In [13]:
gs = generate_scenario()
# Load official dictionaries
# col_names option allow you to algin your column names the project dictionary.
gs.load_offical_icd("cim_2024.xlsx",col_names={"code" : "icd_code","libelle":"icd_code_description"} )
gs.load_offical_procedures("ccam_actes_2024.xlsx",col_names={"code":"procedure","libelle_long":"procedure_description"} )
col_names={"Code CIM":"icd_parent_code","Localisation":"primary_site","Type Histologique":"histological_type",
           "Stade":"stage","Marqueurs Tumoraux":"biomarkers","Traitement":"treatment_recommandation","Protocole de Chimiothérapie":"chemotherapy_regimen"}
gs.load_cancer_treatement_recommandations("Tableau récapitulatif traitement cancer.xlsx",col_names ) 
col_names={"racine":"drg_parent_code","lib_spe_uma":"specialty","ratio_spe_racine":"ratio"}
gs.load_specialty_refential("dictionnaire_spe_racine.xlsx",col_names)
gs.load_referential_hospital("chu")
gs.load_exclusions("exclusions")

# Load data from BN  PMSI
col_names={"racine":"drg_parent_code","das": "icd_secondary_code","diag":"icd_primary_code","categ_cim":"icd_primary_parent_code",
            "mdp":"case_management_type","nb_situations":"nb","acte":"procedure",
            "mode_entree":"admission_mode",
            "mode_sortie":"discharge_disposition",
            "mode_hospit":"admission_type"}
gs.load_classification_profile("bn_pmsi_cases_20250819.csv", col_names)
gs.load_secondary_icd("bn_pmsi_related_diag_20250818.csv",col_names)
gs.load_procedures("bn_pmsi_procedures_20250818.csv",col_names)

In [14]:
cols_scenario = ["first_name","last_name","cage2","cage","sexe",
                "last_name_med","icd_primary_code",
                "admission_type","admission_mode","discharge_disposition",
                'drg_parent_code','icd_primary_code','icd_secondaray_code','cd_md_pec']
                
cols_cancer = ["cancer_stage","TNM_score","histological_type","treatment_recommandation","chemotherapy_regimen"]



In [8]:
#Prepare cases
df_profile = gs.df_classification_profile.drop(columns="nb")
df_profile = df_profile[df_profile.icd_primary_code.isin(gs.icd_codes_cancer)]

In [233]:
df_scenario =[]
for i in range(0,5):

    current_profile = df_profile.iloc[i,:]

    scenario = gs.generate_scenario_from_profile(current_profile)
    row = {k:scenario[k] for k in scenario if k in cols_scenario }
    cancer = [scenario[k] for k in scenario if k in cols_cancer ]

    row.update({"cancer":cancer})
    
    case  = gs.make_prompts_marks_from_scenario(scenario)
    
    row.update({'case': case})


    if row['admission_type'] == "Inpatient" and row['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_complete.txt"
    elif row['admission_type'] == "Outpatient" and row['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_outpatient.txt"
    elif row['drg_parent_code'][2:3]=="K" :
        template_name = "interventionnel.txt"
    elif row['cd_md_pec']==17 :
        template_name = "bilan.txt"
    else:
        template_name = "scenario_onco_v1.txt"
        
    prompt =  prepare_prompt("templates/" + template_name ,case =case)
    row.update({'prompt': prompt})

    df_scenario.append(row)

C:\Users\3056269\AppData\Roaming\Python\Python310\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\3056269\AppData\Roaming\Python\Python310\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


In [8]:
rules["D3-3"]['texte']

'HOSPITALISATION POUR DIAGNOSTIC. Séjour programmé pour bilan pré-oératoire\n'

In [9]:
"D3-3" in rules

True

In [ ]:
def choose_atih_hierarchy_rule(self,icd_primary_code, case_management_type=None, icd_secondaray_code=None, procedure=None, admission_type = None, drg_parent_code = None):
        """
        Détermine la situation clinique et la règle ATIH applicable en fonction des paramètres d'entrée.

        Args:
            selficd_primary_code (str): Diagnostic principal (code CIM-10).
            icd_related_code (str): Diagnostic relié (code CIM-10 ou None).
            das (list): Liste des diagnostics associés (codes CIM-10 ou None).
            type_hosp (str): "HC" (hospitalisation complète) ou "HP" (hospitalisation partielle).
            ds (int): Durée de séjour (en jours).

        Returns:
            dict: Un dictionnaire contenant la situation clinique et la règle ATIH applicable.
        """
        # Liste des règles et situations possibles
        rules_id = []

        # --- Règles liées à l'hospitalisation pour diagnostic ---
        # Règle D1 : DP = affection causale, DR = vide
        # On ajoute aussi la situation de pancréatite aigue, qui peut être un épdisode unique (règle D1) 
        # ou compliquer une pancréatite chronique (regle D5)

        icd_primary_description = gs.get_icd_description(icd_primary_code)

        if case["icd_primary_code"] :
            rules_id.append(["D1"] )

        # Règle D2 : DP = symptomatologie, DR = None ou suspicion non confirmée
        if icd_primary_code.startswith('R') or icd_primary_code.startswith('Z03'):
            rules_id +=  ["D2"] 

        # Règle D3 : DP = Z04.800, Z04.801, Z04.802, Z01.5
        if icd_primary_code in self.icd_codes_overnight_study:
           rules_id += ["D3-1"] 
        
        if icd_primary_code in self.icd_codes_sensitization_tests:
           rules_id  += ["D3-2"]    
        
        if icd_primary_code in self.icd_codes_preoperative_assessment:
           rules_id  += ["D3-3"]   

        # Règle D4 : DP = motif de recours pour antécédent ou symptomatologie
        if icd_primary_code in self.icd_codes_personnel_history + self.icd_codes_family_history :
            rules_id  += ["D4"]   

        # Règle D5 : DP = maladie chronique en poussée aiguë
        if icd_primary_code in self.icd_code_chronic :
            rules_id += ["D5"] 

        # Règle D6 : DP = code spécifique pour poussée aiguë (ex. : J44.1 pour BPCO)
        if icd_primary_code in self.icd_codes_chronic_attack:
            rules_id  += ["D6"] 

        # Règle D7 : DP = complication d’une maladie chronique
        # if not icd_primary_code.startswith('Z') and (icd_related_code is not None and (icd_related_code.startswith('E') or icd_related_code.startswith('I') or icd_related_code.startswith('J') or icd_related_code.startswith('N'))):
        if icd_primary_code in self.icd_codes_chronic_complications : 
            #We can add a test on icd_secondary that should contain a least one chronicle disease.
            rules_id  += ["D7"] 

        # Règle D8 : DP = affection intercurrente indépendante
        if not icd_primary_code.startswith('Z') and (icd_primary_code not in self.icd_codes_chronic) and not set(das).isdisjoint(self.icd_codes_chronic):
            rules_id  += ["D8"]  

        # Règle D9 : DP = tumeur maligne (bilan initial d’extension)
        if icd_primary_code in self.icd_codes_cancer:
            rules_id  += ["D9"]  

        # --- Règles liées à l'hospitalisation pour traitement ---
        # Règle T1 : DP = code Z pour traitement répétitif
        if case_management_type in  self.icd_codes_sessions:
            rules_id  += ["T1"]  

        # Règle T2 : Exceptions à T1 (douleur chronique, ascite, épanchement pleural, toxine botulique)
        if icd_primary_code in self.icd_codes_t2_chronic_intractable_pain:
            rules_id  += ["T2-R52"]  
        if icd_primary_code in self.icd_codes_ascites:
            rules_id  += ["T2-R18"]  
        if icd_primary_code in self.icd_codes_pleural_effusion:
            rules_id  += ["T2-J9"]  
        if procedure in self.procedure_botulic_toxin and admission_type ="Outpatient":
            rules_id  += ["T2-Toxine"]  
           
        # Règle T3 : Traitement chirurgical unique
        if drg_parent_code[2:3] in ["C"] :

            rules_id  += ["T3"]

        # Règle T4 : Chirurgie esthétique
        if case_management_type in self.icd_codes_cosmetic_surgery:
            rules_id  += ["T4"]

        # Règle T5 : Chirurgie plastique non esthétique
        if case_management_type in self.icd_codes_plastic_surgery:
            rules_id  += ["T5"]

        # Règle T6 : Intervention de confort
        if case_management_type in icd_codes_comfort_intervention:
            rules_id  += ["T5"]

        # Règle T7 : Soins spécifiques de stomies/prothèses
        if icd_primary_code.startswith('Z4') and icd_primary_code not in ["Z41.0", "Z41.1", "Z41.80"]:
            situations.append({
                "situation": "Séjour pour soins spécifiques de stomies/prothèses (DP = code Z43 à Z47 ou Z49).",
                "regle": "T7"
            })

        # Règle T8 : Traitement interventionnel
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds <= 3:
            situations.append({
                "situation": "Séjour pour traitement interventionnel (DP = maladie traitée).",
                "regle": "T8"
            })

        # Règle T9 : Traitement médical unique
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds <= 5:
            situations.append({
                "situation": "Séjour pour traitement médical unique (DP = affection traitée).",
                "regle": "T9"
            })

        # Règle T10 : Curiethérapie ou injection de fer
        if icd_primary_code in ["Z51.01", "Z51.2"]:
            situations.append({
                "situation": "Séjour pour curiethérapie ou injection de fer (DP = code Z51).",
                "regle": "T10"
            })

        # Règle T11 : Soins palliatifs
        if icd_primary_code == "Z51.5":
            situations.append({
                "situation": "Séjour pour soins palliatifs (DP = Z51.5).",
                "regle": "T11"
            })

        # Règle T12 : Accouchement normal
        if icd_primary_code == "O80.0":
            situations.append({
                "situation": "Séjour pour accouchement normal (DP = O80.0).",
                "regle": "T12"
            })

        # Règle T13 : Nouveau-né en maternité
        if icd_primary_code.startswith('Z38'):
            situations.append({
                "situation": "Séjour pour nouveau-né en maternité (DP = code Z38).",
                "regle": "T13"
            })

        # Règle T14 : Reprise de traitement chirurgical
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds > 7:
            situations.append({
                "situation": "Séjour pour reprise de traitement chirurgical (DP = maladie traitée).",
                "regle": "T14"
            })

        # Règle T15 : Intervention prophylactique
        if icd_primary_code == "Z40":
            situations.append({
                "situation": "Séjour pour intervention prophylactique (DP = Z40).",
                "regle": "T15"
            })

        # --- Règles liées à l'hospitalisation pour surveillance ---
        # Règle S1 : Surveillance négative
        if icd_primary_code.startswith('Z') and icd_primary_code not in ["Z94", "Z95", "Z39.08", "Z76.2"]:
            situations.append({
                "situation": "Séjour pour surveillance négative (DP = code Z).",
                "regle": "S1"
            })

        # Règle S2 : Surveillance post-transplantation
        if icd_primary_code.startswith('Z94'):
            situations.append({
                "situation": "Séjour pour surveillance post-transplantation (DP = code Z94).",
                "regle": "S2"
            })

        # Règle S3 : Surveillance d’implant cardiovasculaire
        if icd_primary_code.startswith('Z95'):
            situations.append({
                "situation": "Séjour pour surveillance d’implant cardiovasculaire (DP = code Z95).",
                "regle": "S3"
            })

        # Règle SD1 : Surveillance positive
        if icd_primary_code.startswith('Z') and icd_related_code is not None and not icd_related_code.startswith('Z'):
            situations.append({
                "situation": "Séjour pour surveillance positive (DP = code Z, DR = affection nouvelle).",
                "regle": "SD1"
            })

        # Règle SD2 : Récidive de cancer
        if icd_primary_code.startswith('C') and icd_related_code is not None and icd_related_code.startswith('Z85'):
            situations.append({
                "situation": "Séjour pour récidive de cancer (DP = tumeur récidivante, DR = antécédent de cancer).",
                "regle": "SD2"
            })

        # Règle S4 : Surveillance post-accouchement
        if icd_primary_code in ["Z39.08", "Z76.2"]:
            situations.append({
                "situation": "Séjour pour surveillance post-accouchement ou nouveau-né sain (DP = code Z39 ou Z76).",
                "regle": "S4"
            })

        # --- Règles pour plusieurs DP possibles ---
        # Règle M1 : Un problème principal
        if len(situations) > 1:
            situations.append({
                "situation": "Plusieurs problèmes de santé, mais un problème principal mobilise l’essentiel des efforts de soins.",
                "regle": "M1"
            })

        # Règle M2 : Efforts équivalents
        if len(situations) > 1:
            situations.append({
                "situation": "Plusieurs problèmes de santé avec efforts de soins équivalents.",
                "regle": "M2"
            })

        # --- Tirage au sort si plusieurs situations possibles ---
        if len(situations) == 0:
            return {"situation": "Aucune règle ATIH ne correspond à ce scénario.", "regle": None}
        elif len(situations) == 1:
            return situations[0]
        else:
            return random.choice(situations)

    # Exemple d'utilisation
    if __name__ == "__main__":
        # Exemple 1 : DP = Z04.800, DR = G40.1
        print(choisir_regle_atih(icd_primary_code="Z04.800", icd_related_code="G40.1"))

        # Exemple 2 : DP = I21.9, DR = R07.4
        print(choisir_regle_atih(icd_primary_code="I21.9", icd_related_code="R07.4"))

        # Exemple 3 : DP = C50.9, DR = None
        print(choisir_regle_atih(icd_primary_code="C50.9", icd_related_code=None))

        # Exemple 4 : DP = Z51.5, DR = None
        print(choisir_regle_atih(icd_primary_code="Z51.5", icd_related_code=None))

In [15]:
df_res = pd.read_csv("results/test_generation_v1_response.csv")
    

In [16]:
df_res.columns


Index(['Unnamed: 0', 'age', 'sexe', 'date_entry', 'date_discharge',
       'date_of_birth', 'first_name', 'last_name', 'icd_primary_code',
       'case_management_type', 'icd_secondaray_code', 'admission_mode',
       'discharge_disposition', 'cancer_stage', 'score_TNM',
       'histological_type', 'treatment_recommandation', 'chemotherapy_regimen',
       'drg_parent_code', 'cage', 'cage2', 'admission_type', 'dms', 'los_mean',
       'los_sd', 'drg_parent_description', 'icd_parent_code',
       'icd_primary_description', 'case_management_type_description',
       'first_name_med', 'last_name_med', 'text_secondary_icd_official',
       'procedure', 'text_procedure', 'case_management_type_text', 'cd_md_pec',
       'prompt', 'biomarkers', 'response'],
      dtype='object')

In [20]:

df_res

In [34]:
i = 6
print(df_res.loc[i,"prompt"])
print(json.loads(df_res.loc[i,"response"] )['CR'])

Vous êtes un oncologue clinicien expert. Votre tâche est de générer un compte rendu d'hospitalisation en style clinique synthétique.


**SCÉNARIO DE DÉPART :**
- Âge du patient : 69 ans
- Sexe du patient : Féminin
- Date d'entrée : 10/01/2024
- Date de sortie : 10/01/2024
- Date de naissance : 26/07/1954
- Prénom du patient : Nazra
- Nom du patient : Velay
- Mode de prise en charge : Hospitalisation en ambulatoire pour Examen de contrôle après d'autres traitements pour tumeur maligne
- Codage CIM10 :
   * Diagnostic principal : Tumeur maligne du sein (C50)
   * Diagnostic relié : Examen de contrôle après d'autres traitements pour tumeur maligne (Z088)
   * Diagnostic associés : 
- Autres formes de schizophrénie (F208)
- Cachexie (R64)
- Tumeur maligne secondaire du foie et des voies biliaires intrahépatiques (C787)
- Tumeur maligne secondaire du cerveau et des méninges cérébrales (C793)
- Douleurs chroniques irréductibles, autres et non précisées (R5218)
- Pneumopathie bactérienne, san

In [22]:
gs.df_icd_official[(gs.df_icd_official.icd_code.str.contains("Z015")) ].rename(columns={"icd_code":"code","icd_code_description":"description"}).to_csv( path_ref + "icd_codes_sensitization_tests.csv",index=False,sep=";")

In [23]:
 pd.read_csv(path_ref + "icd_codes_sensitization_tests.csv",sep=";").code

0    Z015
Name: code, dtype: object

In [ ]:
|Z94|Z951|Z952|Z953|Z954|Z955|Z956|Z957|Z958
|
Z762

0      E055
1      E100
2      E101
3     E1100
4     E1108
5     E1110
6     E1118
7      J440
8      J441
9       J46
10     K850
11     K851
12     K852
13     K853
14     K858
15     K859
Name: code, dtype: object

In [3]:
"toto" in []

False

In [6]:
df_scenario = pd.read_csv("results/generated_scenarios_20250918_1017.csv")

In [ ]:
df_scenario.to_csv("results/generated_scenarios_20250918_1017.csv",index=False)

In [7]:
print( "### Motif d’hospitalisationMonsieur Alann Vervoitte, 59 ans, est admis aux urgences de l’Hôpital Larrey le 10/11/2024 pour un tableau fébrile à 39,5°C évoluant depuis 48h, associé à des myalgies diffuses, des céphalées intenses et une coloration jaune des conjonctives. Il rapporte également des douleurs lombaires et une oligurie. À l’interrogatoire, il évoque un contact récent avec des eaux stagnantes (pêche en étang la semaine précédente). L’examen initial retrouve une hypotension à 90/60 mmHg, une tachycardie à 110 bpm et des pétéchies disséminées. Une suspicion de leptospirose sévère est évoquée en urgence, motivant son transfert en unité d’hépatogastroentérologie pour prise en charge diagnostique et thérapeutique.---### AntécédentsSur le plan médical, le patient présente :- Un **cancer de la prostate** diagnostiqué en 2022 (stade localisé, traité par radiothérapie externe en 2023, actuellement en surveillance avec PSA à 1,2 ng/mL en octobre 2024).- Des **troubles du rythme cardiaque** (bloc auriculoventriculaire de type Mobitz I diagnostiqué en 2021, sans pose de pacemaker à ce jour).- Une **dépendance alcoolique chronique** (consommation quotidienne estimée à 60 g/j d’éthanol pur, plusieurs tentatives de sevrage infructueuses).- Des **carences en magnésium** récidivantes (dernier épisode en 2023, lié à une malnutrition et à l’alcoolisme).- Un **érysipèle** de la jambe droite en 2020, traité par antibiothérapie per os.Aucun antécédent chirurgical notable en dehors de l’adénectomie prostatique par voie coelioscopique en 2023. Pas d’allergie médicamenteuse connue. Antécédents familiaux sans particularité (pas de cancer prostatique connu).---### Mode de vieAncien ouvrier du bâtiment (retraité depuis 2023 pour invalidité liée à son cancer), le patient vit seul dans un pavillon périurbain. Il déclare une **consommation alcoolique ancienne** (bière et vin quotidiennement depuis l’âge de 20 ans), sans autre toxique. Tabagisme sevré en 2020 (20 paquets-année). Activité physique réduite (marche occasionnelle). Alimentation déséquilibrée, pauvre en fruits et légumes, avec épisodes de dénutrition en lien avec ses hospitalisations passées.Aucun suivi social régulier, mais bénéficiaire de l’AME pour ses soins. Pas de soutien familial proche (divorcé, 2 enfants en région parisienne avec contacts sporadiques).---### Histoire de la maladieLes symptômes ont débuté 72h avant l’admission par un syndrome grippal (frissons, courbatures) avec fièvre mesurée à 38,5°C. Le 09/11, apparition d’une **ictère conjonctival** puis cutané, associée à des douleurs musculaires diffuses (notamment aux mollets) et une asthénie majeure. Le patient note aussi une **diminution du volume des urines** (diurèse estimée à 500 mL/24h) et des saignements gingivaux au brossage. Il consulte son médecin traitant qui suspecte une **infection sévère** et l’adresse aux urgences.À l’arrivée, le bilan initial confirme une **insuffisance rénale aiguë** (créatinine à 220 µmol/L vs 90 µmol/L en 2023), une **cytolyse hépatique** (ALAT à 5xN, bilirubine totale à 80 µmol/L) et une **thrombopénie** à 80 G/L. La notion de baignade en eau douce récente oriente vers une **leptospirose ictéro-hémorragique**, d’autant que le patient n’a pas voyagé en zone tropicale.---### Examen cliniqueÀ l’admission (10/11/2024, 14h) :- **Constantes** : TA 90/60 mmHg (vs 130/80 mmHg habituelle), FC 110 bpm (rythme régulier), FR 20/min, SpO₂ 98% en air ambiant, T° 39,2°C.- **Poids** : 68 kg (vs 75 kg en 2023, amaigrissement de 7 kg en 1 an).- **État général** : Patient prostré, déshydraté (pli cutané), ictère généralisé. Score ECOG estimé à 2 (alité >50% du temps depuis 48h).- **Examen physique** :  - **Cutané** : pétéchies des membres inférieurs et du tronc, érythème chaud et douloureux de la face antérieure de la jambe droite (récidive d’érysipèle ?).  - **Cardio** : souffle systolique 2/6 sans irradiation, pas de galop. ECG en urgence : bloc AV du 1er degré (PR à 280 ms) connu, sans aggravation.  - **Abdomen** : souple, foie palpable à 2 cm sous le rebord costal droit (sensible), pas de splénomégalie.  - **Neurologique** : céphalées diffuses, photophobie, pas de signe de localisation. Raideur méningée absente.  - **Urogénital** : prostate indurée, non douloureuse, sans globe vésical.---### Examens complémentaires- **Biologie** (10/11/2024) :  - **NFS** : Hb 11,2 g/dL (normocyte), leucocytes 12 G/L (PNN 80%), plaquettes 80 G/L.  - **Bilan rénal** : urée 15 mmol/L, créatinine 220 µmol/L (DFG estimé à 25 mL/min), kaliémie 5,1 mmol/L, natrémie 130 mmol/L.  - **Bilan hépatique** : bilirubine totale 80 µmol/L (conjuguée 60), ALAT 200 UI/L (N<40), ASAT 180 UI/L, TP 60% (INR 1,5).  - **Inflammatoire** : CRP 220 mg/L, PCT 15 µg/L.  - **Magnésémie** : 0,5 mmol/L (hypomagnésémie sévère).  - **Sérologie leptospirose** : IgM positives (titres élevés), PCR sanguine en attente.  - **Hémocultures** : négatives à J2.  - **BU** : protéinurie 1+, hématies 20/mm³, leucocytes 10/mm³.- **Imagerie** :  - **Échographie abdominale** (11/11) : foie homogène sans dilatation biliaire, reins de taille normale sans obstruction. Vésicule biliaire fine, sans lithiase.  - **ECG** (10/11) : bloc AV 1er degré stable, pas d’ischémie aiguë.  - **Radio thorax** : normale (pas de foyer infectieux).---### Évolution pendant l'hospitalisation#### Démarche diagnostiqueLe tableau clinique et biologique (fièvre + ictère + insuffisance rénale + thrombopénie + exposition à eau douce) est évocateur d’une **leptospirose sévère forme ictéro-hémorragique**. La sérologie positive (IgM) confirme le diagnostic. Les autres hypothèses (hépatite virale, sepsis à germes banals) sont écartées devant :- L’absence d’élévation des transaminases >10N.- Les hémocultures stériles.- La notion d’exposition environnementale typique.Un **érysipèle de la jambe droite** est également diagnostiqué (érythème + fièvre + leucocytose), probablement surinfecté à streptocoque (prélèvement cutané positif au streptocoque β-hémolytique du groupe A).Le **bloc AV** est surveillé en scope (pas d’aggravation sous traitement). La **carença en magnésium** est corrigée par perfusion (sulfate de magnésium).#### Traitements- **Antibiothérapie** :  - **Leptospirose** : céftriaxone 2 g/j IV (J1-J7), relayé par doxycycline 200 mg/j per os (J8-J14).  - **Érysipèle** : amoxicilline-acide clavulanique 1 g x3/j IV (J1-J5), puis per os.- **Traitement symptomatique** :  - Réhydratation IV (sérum physiologique 1,5 L/j) + correction des troubles ioniques (K+, Mg2+).  - Paracétamol 1 g x3/j pour la fièvre et les céphalées.  - Oméprazole 20 mg/j (protection gastrique).- **Prise en charge du cancer de prostate** :  - Surveillance du PSA (stable à 1,2 ng/mL, pas de signe de récidive).  - Consultation d’oncologie programmée en post-hospitalisation pour évaluation du statut hormonal (castration chimique en cours par acétate de leuproreline).- **Alcoolodépendance** :  - Initiation d’un traitement par acamprosate 666 mg x3/j, avec liaison avec le CSAPA local pour un sevrage ambulatoire.  - Prescription de vitamine B1 (thiamine) 250 mg/j IV puis per os.#### Évolution clinique- **J2 (12/11)** : Apyrexie sous antibiothérapie, diurèse redevenue normale (1,5 L/24h), amélioration de l’ictère (bilirubine à 50 µmol/L).- **J3 (13/11)** : Disparition des pétéchies, amélioration de l’érythème de jambe. Créatinine en baisse à 150 µmol/L.- **J5 (15/11)** : Reprise de l’alimentation orale, magnésémie normalisée (0,8 mmol/L). ECG de contrôle : bloc AV inchangé.- **J7 (17/11)** : Sortie prévue avec :  - Antibiotiques per os pour 7 jours supplémentaires.  - Rendez-vous en infectiologie (contrôle sérologique leptospirose à J21).  - Consultation en addictologie sous 1 semaine.---### ConclusionHospitalisation de 7 jours pour une **leptospirose ictéro-hémorragique** compliquée d’une **insuffisance rénale aiguë** et d’un **érysipèle surinfecté**, sur terrain fragilisé par une **alcoolodépendance chronique**, une **hypomagnésémie** et un **antécédent de cancer prostatique**. Évolution favorable sous céftriaxone et réhydratation, avec correction des troubles hydro-électrolytiques. Le **bloc auriculoventriculaire** connu n’a pas évolué. Sortie vers le domicile avec relais ambulatoire organisé (infectiologie, addictologie, oncologie).Le patient est informé des mesures de prévention (éviction des eaux stagnantes, port de bottes pour activités à risque) et de la nécessité d’un sevrage alcoolique pour limiter les récidives infectieuses et métaboliques.")

### Motif d’hospitalisationMonsieur Alann Vervoitte, 59 ans, est admis aux urgences de l’Hôpital Larrey le 10/11/2024 pour un tableau fébrile à 39,5°C évoluant depuis 48h, associé à des myalgies diffuses, des céphalées intenses et une coloration jaune des conjonctives. Il rapporte également des douleurs lombaires et une oligurie. À l’interrogatoire, il évoque un contact récent avec des eaux stagnantes (pêche en étang la semaine précédente). L’examen initial retrouve une hypotension à 90/60 mmHg, une tachycardie à 110 bpm et des pétéchies disséminées. Une suspicion de leptospirose sévère est évoquée en urgence, motivant son transfert en unité d’hépatogastroentérologie pour prise en charge diagnostique et thérapeutique.---### AntécédentsSur le plan médical, le patient présente :- Un **cancer de la prostate** diagnostiqué en 2022 (stade localisé, traité par radiothérapie externe en 2023, actuellement en surveillance avec PSA à 1,2 ng/mL en octobre 2024).- Des **troubles du rythme cardiaqu

In [15]:
file_name = "extractions_associations_20250909.csv"
df_classification_profile = pd.read_csv(gs.path_data + file_name,sep=";")

In [21]:
df_classification_profile

,type_hosp,diagnostic,age,racine,diagnostic_associes
0,HC,A000,ge_18,06M03,B180 B962 E46 E86 F122 I451 N178 R392 R571 T79...
1,HC,A001,ge_18,06M03,A038 E039 M1007 Z290
2,HC,A001,ge_18,06M03,E8718 R571 Z290
3,HC,A001,ge_18,06M03,Z290
4,HC,A009,ge_18,06M03,D695 E780 E8718 E891 I10 I252 I255 I482 K219 K...
...,...,...,...,...,...
2529155,HP,Z998,lt_18,23M20,C410
2529156,HP,Z998,lt_18,23M20,D588
2529157,HP,Z998,lt_18,23M20,K318
2529158,HP,Z998,lt_18,23M20,K500 Z713


In [57]:
df = gs.df_classification_profile.copy()
file_name = "icd_primary_exclusions.csv"
df_exc = pd.read_csv(gs.path_ref + file_name,sep=";")

In [63]:
df_z = df[df.icd_primary_code.str.slice(0,1)=="Z"].groupby("icd_primary_code").agg(nb=("nb","sum")).reset_index().rename(columns={"icd_primary_code":"icd_code"})

In [64]:
df_z = gs.df_icd_official[gs.df_icd_official.icd_code.str.slice(0,1)=="Z"].merge(df_z,how= "left")

In [65]:
df_z = df_z.assign(exclus="Non")
df_z.exclus[df_z.icd_code.isin(df_exc.icd_code)]="Oui"

C:\Users\3056269\AppData\Local\Temp\ipykernel_22508\1599298326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_z.exclus[df_z.icd_code.isin(df_exc.icd_code)]="Oui"


In [67]:
df_z.to_excel("C:/data/wd/z_dp.xlsx",index=False)

In [56]:
df_exc.icd_code

0     Z001
1     Z004
2     Z005
3     Z006
4     Z008
      ... 
92    C118
93    C138
94    C258
95    C328
96    Z491
Name: icd_code, Length: 97, dtype: object

In [ ]:
df_z = gs.df_icd_official[]

In [50]:
gs.df_icd_official.to_excel("C:/data/wd/z_dp.xlsx")

df_exc

,icd_code,description
0,Z001,NaN
1,Z004,NaN
2,Z005,NaN
3,Z006,NaN
4,Z008,NaN
...,...,...
92,C118,NaN
93,C138,NaN
94,C258,NaN
95,C328,NaN


In [ ]:
profile = df_scenario.loc[1]
grouping_procedure =["procedure","drg_parent_code","icd_primary_code","cage2","sexe"]
procedures = gs.sample_from_df(profile =profile,df_values= gs.df_procedures[grouping_procedure],nb=1) 
procedures.procedure.values[0]
procedures.procedure_description_official.values[0]